In [2]:
# Configure Environment 
from  dev.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())



Platform:  dev
Data directory :  /home/jovyan/dev/data


## Imports:

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing
import datetime
import pickle
import json

from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Create sentence embeddings from Elmo via TensorFlow Hub
https://tfhub.dev/google/elmo/1

In [4]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

## Read Plot Sentences

In [5]:
# Change Directory to Data
# read from data
# Create elmoWeights Folder, in not exists
# Write weights to data/emloWeights


# datadir = os.path.join(rootdir, 'data') 
path_to_json = './person/wikipedia/'

elmodir = os.getcwd()+'/models/'

# create elmoWeights directory - if it does not exist
if not os.path.exists(elmodir):
    os.makedirs(elmodir) 

## READ PLOT DATA - ALL from PKL file

In [6]:

json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)


bio_dict = {}
for f in json_files:
    data = json.load(open(f) )
    bio = data["biography"]
    bio_dict[data["_id"]] = bio
    


In [7]:
sentencesAll = list(bio_dict.values())
sentencesAll[0]

"Kevin Michael Costner (born January 18, 1955) is an American actor, director, producer, and musician. His accolades include two Academy Awards, three Golden Globe Awards, one Primetime Emmy Award, and two Screen Actors Guild Awards.\nCostner began his acting career with Sizzle Beach, U.S.A. (1981). Following a few minor supporting parts, he rose to prominence with his  portrayal of Eliot Ness in The Untouchables (1987). This was followed by a successful period in his career with starring roles in No Way Out (1987), Bull Durham (1988), Field of Dreams (1989), Dances with Wolves (1990), for which he won two Academy Awards, JFK (1991), Robin Hood: Prince of Thieves (1991), The Bodyguard (1992), and A Perfect World (1993). In 1995, Costner starred in and co-produced Waterworld. The most expensive film ever made at the time, it was a major box office disappointment which marked a downturn in his career. His second directorial feature The Postman (1997) was another disappointment. He has si

In [8]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        s = s[0:4000]
        sentences[i] = s
        i += 1
    return sentences

In [9]:

# For Pickle Input
sentences = cleanSentences(sentencesAll)

In [10]:
# Test - 
sentences[0]
# len(sentences)

"kevin michael costner (born january 18 1955) is an american actor director producer and musician his accolades include two academy awards three golden globe awards one primetime emmy award and two screen actors guild awards costner began his acting career with sizzle beach usa (1981) following a few minor supporting parts he rose to prominence with his  portrayal of eliot ness in the untouchables (1987) this was followed by a successful period in his career with starring roles in no way out (1987) bull durham (1988) field of dreams (1989) dances with wolves (1990) for which he won two academy awards jfk (1991) robin hood: prince of thieves (1991) the bodyguard (1992) and a perfect world (1993) in 1995 costner starred in and co-produced waterworld the most expensive film ever made at the time it was a major box office disappointment which marked a downturn in his career his second directorial feature the postman (1997) was another disappointment he has since starred in numerous less-su

# Score Sentences in Batches

In [11]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings

In [12]:
# score sentences
def scoreSentences (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

In [13]:
# Save Weights - use names based on Movie 
import pickle as pkl

def saveWeights(elmoFN, x):
    with open('./models/'+elmoFN,'wb') as f: pkl.dump(x, f)
    return

## Score N sentences at a time


In [14]:
%matplotlib inline

from IPython.core.magic import register_line_magic


In [15]:
import math 

batch_size = 100                      # Specify number of sentences per file  (was 500)
numPlots = len(sentences)
numfiles = math.ceil(numPlots/batch_size) 
[os.remove(os.path.join(elmodir,f)) for f in os.listdir(elmodir) if 'elmoBioweights' in f]

start = 0
end = batch_size
for file in range(1,numfiles+1):
    elmoFN = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+batch_size

Score sentences: elmoBioweights0to100.pkl


## Read and Concatenate all weight arrays in elmoWeights to build


In [16]:
a='elmoPlotsweights250to300'
if 'Plot' in a:
    print('pass')

pass


In [17]:
# Read Weights - use names based on actorID 
import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(elmodir+file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
def readWeights(elmodir, modelname):
    noFiles = len([name for name in os.listdir(elmodir) if modelname in name ])
    xin = readElmoWeightFile('elmoBioweights0to'+str(batch_size)+'.pkl')
    size = batch_size
    for x in range(1, noFiles):
        start = x*size
        end = start+size
        filename = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
        print(filename)
        xin += readElmoWeightFile(filename)
    return xin

# Save model and labels catalog.

In [18]:
os.listdir(elmodir)

['actorNameRef.pkl',
 'celebrityBio.json',
 'celebrityLookup.pkl',
 'bio_dict.pkl',
 'elmoPlotsweights0to50.pkl',
 'elmoPlotsweights1000to1050.pkl',
 'elmoPlotsweights100to150.pkl',
 'elmoPlotsweights1050to1100.pkl',
 'elmoPlotsweights1100to1150.pkl',
 'elmoPlotsweights1150to1200.pkl',
 'elmoPlotsweights150to200.pkl',
 'elmoPlotsweights1200to1250.pkl',
 'elmoPlotsweights200to250.pkl',
 'elmoPlotsweights350to400.pkl',
 'elmoPlotsweights300to350.pkl',
 'elmoPlotsweights400to450.pkl',
 'elmoPlotsweights250to300.pkl',
 'elmoPlotsweights450to500.pkl',
 'elmoPlotsweights500to550.pkl',
 'elmoPlotsweights50to100.pkl',
 'elmoPlotsweights650to700.pkl',
 'elmoPlotsweights550to600.pkl',
 'elmoPlotsweights600to650.pkl',
 'elmoPlotsweights750to800.pkl',
 'elmoPlotsweights700to750.pkl',
 'elmoPlotsweights850to900.pkl',
 'elmoPlotsweights900to950.pkl',
 'elmoPlotsweights950to1000.pkl',
 'genreStats.pkl',
 'elmoPlotsweights800to850.pkl',
 'faceLabels.pkl',
 'actorGenreVect.pkl',
 'faceFeatures.pkl',
 '

In [19]:
!ls

cast  data  models  movies  person  __pycache__  settings.py  top100Actors.txt


In [20]:

xin = readWeights(elmodir, 'elmoBio')
print("Elmo Vector Shape {0}, {1}".format( len(xin), len(xin[1])) )

pickle.dump( bio_dict, open( "./models/bio_dict.pkl", "wb" ) )
pickle.dump( xin, open( "./models/elmoBioweights.pkl", "wb" ) )


Elmo Vector Shape 99, 1024


In [21]:
# Load models 

bio_dict = pickle.load( open( './models/bio_dict.pkl', 'rb' ) )
xin = pickle.load( open( './models/elmoBioweights.pkl', 'rb' ) )


In [27]:


topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)

celebrityLookup={}
for i, row in actorsDF.iterrows():
    celebrityLookup.update({ row['Rank'] : row['Full Name']})

celebrityLookup['10']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


'Harrison Ford'

In [23]:
# Load models
pickle.dump( celebrityLookup, open( "./models/celebrityLookup.pkl", "wb" ) )
celebrityLookup = pickle.load( open( './models/celebrityLookup.pkl', 'rb' ) )

In [24]:
# Copy models upto S3
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./models/ s3://' + bucket.strip() + '/models/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

gary-celebrity-content-dev-vpc-08172c22ced39d93c

aws s3 sync ./models/ s3://gary-celebrity-content-dev-vpc-08172c22ced39d93c/models/ 
Completed 149.8 KiB/949.7 KiB (3.9 MiB/s) with 4 file(s) remaining
upload: models/bio_dict.pkl to s3://gary-celebrity-content-dev-vpc-08172c22ced39d93c/models/bio_dict.pkl
Completed 149.8 KiB/949.7 KiB (3.9 MiB/s) with 3 file(s) remaining
Completed 405.8 KiB/949.7 KiB (7.6 MiB/s) with 3 file(s) remaining
Completed 661.8 KiB/949.7 KiB (11.4 MiB/s) with 3 file(s) remaining
Completed 664.6 KiB/949.7 KiB (9.3 MiB/s) with 3 file(s) remaining 
upload: models/celebrityLookup.pkl to s3://gary-celebrity-content-dev-vpc-08172c22ced39d93c/models/celebrityLookup.pkl
Completed 664.6 KiB/949.7 KiB (9.3 MiB/s) with 2 file(s) remaining
Completed 809.6 KiB/949.7 KiB (7.0 MiB/s) with 2 file(s) remaining
upload: models/elmoBioweights.pkl to s3://gary-celebrity-content-dev-vpc-08172c22ced39d93c/models/elmoBioweights.pkl
Completed 809.6 KiB/949.7 KiB (7.0 MiB/s) with 1 file

## Test Elasticsearch to store the vertor and peform cosine similarity score.

In [26]:
ESHOST


'elasticsearch-dev.ocp.bdso.salientcrgt-bdso.com'

In [28]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json

es= Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_index='bios'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index    
_doc= 'bio'

headers={"Content-Type": "application/json"}

In [29]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },  
      "celebrity" : {
        "type" : "keyword"
      },         
      "bio_vector": {
        "type": "dense_vector",
        "dims": len(xin[0])
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bios'}

In [30]:
ids = list(bio_dict.keys())
for i, v in zip(ids, xin):
    _id = str(i)
    #print(_id)
    doc = {
        #_actorID	actor	age	birthdate	gender	height
        "_actorID" : i,
        "celebrity" : celebrityLookup[_id],
        "bio_vector": [float(x) for x in v], 
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

{"_index":"bios","_type":"_doc","_id":"78","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":0,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"100","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":1,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"99","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":2,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":3,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"89","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"10","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":5,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"55","_version":1,"result":"created","

{"_index":"bios","_type":"_doc","_id":"58","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":62,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"92","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":63,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"42","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":64,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"38","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":65,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"4","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":66,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"41","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":67,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"36","_version":1,"result":"creat

In [31]:
es.indices.stats(index=_index)

{'_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_all': {'primaries': {'docs': {'count': 99, 'deleted': 0},
   'store': {'size_in_bytes': 2095083},
   'indexing': {'index_total': 99,
    'index_time_in_millis': 122,
    'index_current': 0,
    'index_failed': 0,
    'delete_total': 0,
    'delete_time_in_millis': 0,
    'delete_current': 0,
    'noop_update_total': 0,
    'is_throttled': False,
    'throttle_time_in_millis': 0},
   'get': {'total': 0,
    'time_in_millis': 0,
    'exists_total': 0,
    'exists_time_in_millis': 0,
    'missing_total': 0,
    'missing_time_in_millis': 0,
    'current': 0},
   'search': {'open_contexts': 0,
    'query_total': 0,
    'query_time_in_millis': 0,
    'query_current': 0,
    'fetch_total': 0,
    'fetch_time_in_millis': 0,
    'fetch_current': 0,
    'scroll_total': 0,
    'scroll_time_in_millis': 0,
    'scroll_current': 0,
    'suggest_total': 0,
    'suggest_time_in_millis': 0,
    'suggest_current': 0},
   'merges': {'current': 0,

In [32]:
r = requests.get(esIndex+'/_doc/1')
pprint(r.text)

('{"_index":"bios","_type":"_doc","_id":"1","_version":1,"_seq_no":3,"_primary_term":1,"found":true,"_source":{"_actorID": '
 '"1", "celebrity": "Robert Downey Jr.", "bio_vector": [0.1265835016965866, '
 '-0.08057540655136108, 0.06155654788017273, -0.08581449091434479, '
 '0.17603552341461182, 0.14170533418655396, 0.11070732772350311, '
 '0.02038724347949028, 0.3400860130786896, -0.021635795012116432, '
 '0.12881015241146088, -0.06209710240364075, 0.052605848759412766, '
 '0.1479257196187973, -0.16013342142105103, 0.04454837739467621, '
 '0.2529219388961792, -0.008337859995663166, 0.019959663972258568, '
 '-0.24373601377010345, 0.06672684848308563, 0.2746504247188568, '
 '0.04575183242559433, -0.014501774683594704, 0.28113457560539246, '
 '-0.1338174194097519, 0.006053569260984659, 0.08605078607797623, '
 '0.22848202288150787, 0.2674378752708435, 0.2575271725654602, '
 '-0.03687568008899689, -0.06944053620100021, -0.11197268217802048, '
 '-0.14673154056072235, 0.06052112206816673, -0.1

In [33]:
vec = json.loads(r.text)["_source"]["bio_vector"]
q={
  "query": {
    "script_score": {
      "query" : {
        "match_all" : {}
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, doc['bio_vector']) + 1.0", 
        "params": {
          "query_vector": vec
        }
      }
    }
  }
}

res= es.search(index=_index, body=q)

pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'bios',
                    '_score': 2.0,
                    '_source': {'_actorID': '1',
                                'bio_vector': [0.1265835016965866,
                                               -0.08057540655136108,
                                               0.06155654788017273,
                                               -0.08581449091434479,
                                               0.17603552341461182,
                                               0.14170533418655396,
                                               0.11070732772350311,
                                               0.02038724347949028,
                                               0.3400860130786896,
                                               -0.021635795012116432,
                                               0.12881015241146088,
                          

                                               -0.16689929366111755,
                                               0.5856904983520508,
                                               -0.23556941747665405,
                                               0.5854151844978333,
                                               -0.18758656084537506,
                                               0.11859574168920517,
                                               -0.16512088477611542,
                                               -0.3922484517097473,
                                               -0.27201220393180847,
                                               0.1793106198310852,
                                               -0.00301873660646379,
                                               0.0929868221282959,
                                               0.010721458122134209,
                                               0.18258191645145416,
                                             

                                               0.15753579139709473,
                                               0.3274461328983307,
                                               -0.1465839296579361,
                                               -0.144024059176445,
                                               -0.05623234435915947,
                                               0.05392860248684883,
                                               0.3694725036621094,
                                               -0.073668472468853,
                                               -0.2697048485279083,
                                               -0.29453474283218384,
                                               0.21090294420719147,
                                               0.0023073172196745872,
                                               0.1476643681526184,
                                               0.16024373471736908,
                                               -0

                                               -0.1267196089029312,
                                               -0.020147765055298805,
                                               -0.23800596594810486,
                                               0.07000873237848282,
                                               -0.1760127693414688,
                                               -0.508135974407196,
                                               0.1430063545703888,
                                               0.01176506094634533,
                                               -0.015494940802454948,
                                               0.2665741741657257,
                                               -0.24863296747207642,
                                               -0.13940919935703278,
                                               -0.019971685484051704,
                                               -0.11524845659732819,
                                         

                                               0.35064491629600525,
                                               -0.04437626153230667,
                                               -0.5085781216621399,
                                               0.09677670150995255,
                                               0.042321428656578064,
                                               -0.14869527518749237,
                                               0.16022102534770966,
                                               0.037723783403635025,
                                               0.11543790996074677,
                                               -0.061870913952589035,
                                               0.21327976882457733,
                                               -0.04662902280688286,
                                               0.2072739452123642,
                                               0.07919535040855408,
                                          

                                               -0.3383800685405731,
                                               -0.17096683382987976,
                                               0.021229391917586327,
                                               -0.21018704771995544,
                                               0.18976254761219025,
                                               0.22524097561836243,
                                               -0.1539766490459442,
                                               0.2828201353549957,
                                               0.03407013788819313,
                                               0.01843593828380108,
                                               0.045494548976421356,
                                               -0.1270962804555893,
                                               -0.042864322662353516,
                                               0.05997157841920853,
                                           

In [34]:
[hit['_source']['_actorID'] for hit in res['hits']['hits']]

['1', '12', '5', '65', '55', '48', '61', '6', '66', '52']

# REDUCE DIMENSIONS

### Visualize the sentence arrays in reduced Dimensions using PCA and TSNE

Use PCA and t-SNE to reduce the 1,024 dimensions which are output from ELMo down to 2 so that we can review the outputs from the model.

In machine learning, dimensionality reduction is the process of reducing the number of random variables under consideration by obtaining a set of principal variables. By reducing the dimension of your feature space, you have fewer relationships between features to consider which can be explored and visualized easily and also you are less likely to overfit your model. https://www.datacamp.com/community/tutorials/introduction-t-sne

#### Principal Component Analysis or PCA 
Principal Component Analysis (PCA)is a linear feature extraction technique. It performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized. It does so by calculating the eigenvectors from the covariance matrix. The eigenvectors that correspond to the largest eigenvalues (the principal components) are used to reconstruct a significant fraction of the variance of the original data.

In simpler terms, PCA combines your input features in a specific way that you can drop the least important feature while still retaining the most valuable parts of all of the features. As an added benefit, each of the new features or components created after PCA are all independent of one another.
t-Distributed Stochastic Neighbor Embedding (t-SNE)

#### t-Distributed Stochastic Neighbor Embedding (t-SNE) 
is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing.
